In [1]:
library(sf)
library(ggplot2)
library(tidyverse)
library(tibble)
library(magrittr)
library(redist)
library(sp)
library(spData)
library(spdep)
library(igraph)

va_2015_file <- "C:/Users/madie/OneDrive/data/pre-redist/VA_precinct_2015/VA_precinct_2015.shp"
df <- st_read(va_2015_file)

Linking to GEOS 3.8.0, GDAL 3.0.4, PROJ 6.3.1

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v tibble  3.0.4     v dplyr   1.0.2
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0
v purrr   0.3.4     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract



Attaching package: 'igraph'


The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union


The following objects are masked from 'package:purrr':

    compose, simplify


The following object is masked from 'package:tidyr':

    crossing


The following object is masked from 'package:tibble':

    as_data_frame


The following objects are masked from 'package:stats':

    decom

Reading layer `VA_precinct_2015' from data source `C:\Users\madie\OneDrive\data\pre-redist\VA_precinct_2015\VA_precinct_2015.shp' using driver `ESRI Shapefile'
Simple feature collection with 2577 features and 14 fields
geometry type:  MULTIPOLYGON
dimension:      XYZ
bbox:           xmin: 9963623 ymin: 6151696 xmax: 12442960 ymax: 7217011
z_range:        zmin: 0 zmax: 0
projected CRS:  NAD83 / Virginia North (ftUS)


In [2]:
# list of lists of precincts adjacent to each precincts
adj <- redist.adjacency(df)

In [3]:
# area of each precincts
area <- sf::st_area(df)

In [4]:
# coordinate of centroid of each precinct
centers <- sf::st_coordinates(sf::st_centroid(df))

Warning message in st_centroid.sf(df):
"st_centroid assumes attributes are constant over geometries of x"


In [5]:
crsg_out <- redist.crsg(adj.list = adj, population = df$pop, area = area, x_center = centers[,1], y_center = centers[,2], ndists = 100, thresh = .3)

In [ ]:
# generate 100 random colors that are distinct so that the different districts are distinguishable. 
library(randomcoloR)
colors <- distinctColorPalette(k=100)

In [ ]:
cds <- crsg_out$district_membership
plot <- redist.map(shp=df, 
        district_membership=cds, 
        centroids=FALSE, 
        edges=FALSE,
        title="100 Districts in VA HOD: 2015 CRSG") +
        theme(legend.position="right") +
        scale_fill_manual(values = colors)
        # geom_sf_label(aes(label = cds))
#ggsave("fairfax_10.png")
plot

In [ ]:
library(pscl)
# compute the percent of votes won by democrats in each district under DVS
plot_sv <- redist.metrics(cds, measure = c("DSeats", "DVS"), df$repvote, dvote = df$demvote) %>% 
           # extract DVS into vector
           pull(DVS) %>%
           # calculate seats-votes curve using uniform partisan swing
           seatsVotes(desc="Democratic Vote Shares, VA HOD 2015 CRSG") %>%
           # plot the seats-votes curve
           plot(type="seatsVotes")